In [2]:
import numpy as np
import pandas as pd
import mysql.connector as db
from string import Template

In [39]:
#config
#all cols of db_one should occur first in cols of db_two
#extra data for extra cols in db_two that are not the part of db_one must be provided
config = {
    "db_one_host":"<host>",
    "db_one_user":"<user>",
    "db_one_pass":"<password>",
    "db_one_name":"<db name>",
    "db_one_tble":"<table name>",
    "db_one_cols":["col1","col2",....],
    "db_two_host":"<host>",
    "db_two_user":"<user>",
    "db_two_pass":"<password>",
    "db_two_name":"<db name>",
    "db_two_tble":"<table>",
    "db_two_cols":["col1","col2","col3",....],
    "extra_from_csv":"<Boolean>",
    "csv_file_name":"<file_name>.csv"
}

In [6]:
def get_db_one():
    try:
        my_db = db.connect(host=config["db_one_host"],user=config["db_one_user"],passwd=config["db_one_pass"],database=config["db_one_name"])
        return my_db
    except Exception as e:
        print("error->",e);
        return None

In [7]:
def get_db_two():
    try:
        my_db = db.connect(host=config["db_two_host"],user=config["db_two_user"],passwd=config["db_two_pass"],database=config["db_two_name"])
        return my_db
    except Exception as e:
        print("error->",e);
        return None

In [45]:
def get_extra_data_from_csv():
    try:
        df = pd.read_csv(config["csv_file_name"])
        return df
    except Exception as e:
        print("error -> ",e);
        return None

In [67]:
def get_sql():
    try:
        #fetching data
        sql = Template('SELECT * FROM $table_name')
        sql = sql.substitute({"table_name":config["db_one_tble"]})
        db_one = get_db_one()
        if db_one == None:
            print("error occured in db one")
            return None
        cursor_one = db_one.cursor()
        cursor_one.execute(sql)
        my_data = cursor_one.fetchall()
        
        #handling extra col data
        extra_cols = len(config["db_two_cols"])-len(config["db_one_cols"])
        my_extra_data = None
        if(config["extra_from_csv"]):
            my_extra_data = get_extra_data_from_csv()
        else:
            my_extra_data = []
        
        data_len = min(len(my_data),len(my_extra_data))
        
        #message
        if data_len < len(my_data):
            print("some rows from db one can't be inserted due to less data")
        elif data_len < len(my_extra_data):
            print("some extra data can't be inserted due to less data in db one")
        
        #creating sql for insert
        sql = Template('INSERT INTO $table_name $col_name VALUES $values ')
        col_name = '('+','.join(config["db_two_cols"])+')'
        
        #binder in sal
        binder = np.array(['%s']*len(config["db_two_cols"]))
        binder = '('+','.join(binder)+')'
        
        #values binder array
        values = np.array([binder]*data_len);
        
        #binding values
        bind = []
        for i in range(data_len):
            row = my_data[i]
            for val in row:
                bind.append(val)
            
            #binding extra data
            if extra_cols > 0:
                cols = config["db_two_cols"][len(row):]
                for col in cols:
                    bind.append(my_extra_data[col][i])
        values = ','.join(values)
        sql = sql.substitute({"table_name":config["db_two_tble"],"col_name":col_name,"values":values})
        bind = tuple(bind)
        return [sql,bind]
    except Exception as e:
        print("error get sql -> ",e)
        return None

In [62]:
def migrate_data():
    try:
        [sql,bind] = get_sql()
        db_two = get_db_two();
        if(db_two == None):
            print("Failed to connect to second db")
            return None
        db_cursor = db_two.cursor()
        db_cursor.execute(sql,bind);
        db_two.commit();
        return db_cursor.rowcount
    except Exception as e:
        print("error migrate -> ",e);
        return None

In [68]:
migrate_data()

some rows from db one can't be inserted due to less data


46